In [14]:
#Inventory 2022 auxiliar Cruise Processor
#Libraries
from core.libs import pd, tqdm, Path, json, re
from core.paths import resolve_inventory_paths

In [15]:
from core.libs import pd, tqdm, Path, re
import os

# --- Cambia el país aquí ---
specific_file = "mx"  # Cambia a "gt" o "mx" según el país

base = r"C:\Users\HeyCe\World Tree Technologies Inc\Operations - Documentos\WorldTreeSystem\AuxiliaryScripts\2022"
cheatsheet_path = os.path.join(base, f"{specific_file}_2022_cheatseet.csv")

if not os.path.exists(cheatsheet_path):
    raise Exception(f"No se encontró cheatsheet para {specific_file}: {cheatsheet_path}")

df_cheat = pd.read_csv(cheatsheet_path)

df_valid = df_cheat[
    (df_cheat["TreeFilePath"].notna()) &
    (df_cheat["TreeFilePath"] != "NA") &
    (df_cheat["case"].str.lower() != "missing")
].copy()

print(f"Archivos válidos a procesar: {len(df_valid)}")

def pick_sheet(excel_file):
    sheets = pd.ExcelFile(excel_file).sheet_names
    for sheet in sheets:
        if sheet.lower().strip() in ("input", "sheet1"):
            return sheet
    return sheets[0]

frames = []
for idx, row in tqdm(df_valid.iterrows(), total=len(df_valid), desc="Leyendo archivos"):
    excel_path = Path(row["TreeFilePath"])
    contract_code = row["contractcode"]
    cruise_date = row["inventory_date"]
    try:
        sheet = pick_sheet(excel_path)
        df = pd.read_excel(excel_path, sheet_name=sheet)
        df.columns = [str(c).strip() for c in df.columns]
        df['archivo_origen'] = str(excel_path)
        df['contractcode'] = contract_code
        df['cruise_date'] = cruise_date
        frames.append(df)
    except Exception as e:
        print(f"Error en {excel_path}: {e}")

if not frames:
    raise Exception("❌ Ningún archivo pudo ser leído correctamente.")

df_all = pd.concat(frames, ignore_index=True)
df_all = df_all.dropna(axis=1, how='all')

print(df_all.head(5))


Archivos válidos a procesar: 6


Leyendo archivos: 100%|██████████| 6/6 [00:00<00:00, 10.27it/s]

   Stands::StandNumber  Points::PointNumber  GradingsSerialNumberAtPoint  \
0                    1                    1                            1   
1                    1                    1                            2   
2                    1                    1                            3   
3                    1                    1                            4   
4                    1                    1                            5   

                   Tree_ID    WT Status WT Species    WT Leaf  WT DHT  WT DBH  \
0     54418544659281797120  1 Viva/Vivo   Elongata  4 76-100%       0       0   
1    915487742582450814976  1 Viva/Vivo   Elongata  4 76-100%       0       0   
2  69004127945880611323904  1 Viva/Vivo   Elongata  4 76-100%       0       0   
3  93314542940889307676672  1 Viva/Vivo   Elongata  4 76-100%       0       0   
4       821588747087172992  1 Viva/Vivo   Elongata  4 76-100%       0       0   

      WT THT  WT MHT WT Pests jenkinsTotalAgBiomassGREEN

In [16]:
# --- Exporta el CSV final ---
output = os.path.join(
    base, f"inventory_{specific_file}_2022_concentrado.csv"
)
df_all.to_csv(output, index=False)
print(f"\n✅ CSV concentrado generado en:\n{output}")
print(f"Columnas: {list(df_all.columns)}")
print(f"Filas: {len(df_all)}")




✅ CSV concentrado generado en:
C:\Users\HeyCe\World Tree Technologies Inc\Operations - Documentos\WorldTreeSystem\AuxiliaryScripts\2022\inventory_mx_2022_concentrado.csv
Columnas: ['Stands::StandNumber', 'Points::PointNumber', 'GradingsSerialNumberAtPoint', 'Tree_ID', 'WT Status', 'WT Species', 'WT Leaf', 'WT DHT', 'WT DBH', 'WT THT', 'WT MHT', 'WT Pests', 'jenkinsTotalAgBiomassGREEN', 'GradingsComment', 'Preferences::WorldTreePrefsInventoryType', 'Preferences::WorldTreeUnits', 'Preferences::prefsFileName', 'archivo_origen', 'contractcode', 'cruise_date']
Filas: 1407


In [17]:
#Lo limpié manualmente porque era más fácil lol

In [18]:
#De nuevo armando output
specific_file = "mx"  # Cambia a "gt" o "mx" según el país
output = os.path.join(
    base, f"inventory_{specific_file}_2022_concentrado.csv"
)

In [19]:
from core.libs import pd
from core.schema_helpers import rename_columns_using_schema
from CruisesProcessor.general_processing import process_inventory_dataframe
from CruisesProcessor.general_importer import prepare_df_for_sql, save_inventory_to_sql
from core.db import get_engine
from CruisesProcessor.catalog_normalizer import ensure_catalog_entries


# 1. Lee el CSV limpio (solo columnas de texto, nunca _id)
df = pd.read_csv(output)
df = rename_columns_using_schema(df)

# 🔍 Print para diagnóstico inmediato
print("Columnas después de renombrar:", list(df.columns))

# 🛡️ Blindaje manual: fuerza 'Status' si existe algún alias conocido
if "Status" not in df.columns:
    # Busca cualquier alias que pudiera haber quedado
    posibles = [c for c in df.columns if c.strip().lower().replace(" ", "") in [
        "wtstatus", "estatus", "condicion", "estado", "tree_status", "treestatus"
    ]]
    if posibles:
        print(f"⚠️ Renombrando columna '{posibles[0]}' a 'Status' (falló el renombrado automático)")
        df["Status"] = df[posibles[0]]
    else:
        print("⚠️ 'Status' no encontrado ni con aliases, se crea vacía.")
        df["Status"] = pd.NA


# 2. (Opcional pero recomendable) QA: asegúrate de tener las columnas crudas de catálogo
for col in ["Defect", "Species", "Pests", "Status"]:
    if col not in df.columns:
        print(f"⚠️ WARNING: '{col}' no está, se agrega vacía.")
        df[col] = pd.NA
for c in ["defect_id", "species_id", "pests_id", "status_id"]:
    if c in df.columns:
        print(f"🧹 Eliminando columna '{c}' previa.")
        df = df.drop(columns=[c])

Columnas después de renombrar: ['stand', 'plot', 'tree_number', 'Tree_ID', 'Status', 'Species', 'WT Leaf', 'WT DHT', 'dbh_in', 'tht_ft', 'merch_ht_ft', 'Pests', 'jenkinsTotalAgBiomassGREEN', 'short_note', 'Preferences::WorldTreePrefsInventoryType', 'Preferences::WorldTreeUnits', 'Preferences::prefsFileName', 'archivo_origen', 'contractcode', 'cruise_date']
⚠️ WARNING: 'Defect' no está, se agrega vacía.


In [20]:
print(df.head(5))

   stand  plot  tree_number                  Tree_ID       Status   Species  \
0      1     1            1     54418544659281797120  1 Viva/Vivo  Elongata   
1      1     1            2    915487742582450814976  1 Viva/Vivo  Elongata   
2      1     1            3  69004127945880611323904  1 Viva/Vivo  Elongata   
3      1     1            4  93314542940889307676672  1 Viva/Vivo  Elongata   
4      1     1            5       821588747087172992  1 Viva/Vivo  Elongata   

     WT Leaf  WT DHT  dbh_in     tht_ft  merch_ht_ft Pests  \
0  4 76-100%       0       0   0.984252            0   NaN   
1  4 76-100%       0       0   7.874016            0   NaN   
2  4 76-100%       0       0  11.154856            0   NaN   
3  4 76-100%       0       0   9.842520            0   NaN   
4  4 76-100%       0       0  10.170604            0   NaN   

  jenkinsTotalAgBiomassGREEN short_note  \
0                          ?        NaN   
1                          ?        NaN   
2                      

In [21]:
# 3. Procesa TODO el DataFrame igual que en main.py
engine = get_engine()
country_code = specific_file
from CruisesProcessor.catalog_normalizer import ensure_catalog_entries

# Si falta Disease (como en 2022), créala vacía para evitar el KeyError
if "Disease" not in df.columns:
    print("⚠️ El formulario de 2022 no tiene columna 'Disease', se crea vacía.")
    df["Disease"] = pd.NA
if "Coppiced" not in df.columns:
    print("⚠️ El formulario de 2022 no tiene columna 'Coppiced', se crea vacía.")
    df["Coppiced"] = pd.NA
if "Permanent Plot" not in df.columns:
    print("⚠️ El formulario de 2022 no tiene columna 'Permanent Plot', se crea vacía.")
    df["Permanent Plot"] = pd.NA

df_good, df_bad = process_inventory_dataframe(df, engine, country_code)

print(f"✔️ Filas válidas: {len(df_good)} | ❌ Filas con error: {len(df_bad)}")

if not df_bad.empty:
    df_bad.to_excel("bad_rows_inventory_us_2022.xlsx", index=False)

💻 Conectado a la base de datos helloworldtree
⚠️ El formulario de 2022 no tiene columna 'Disease', se crea vacía.
⚠️ El formulario de 2022 no tiene columna 'Coppiced', se crea vacía.
⚠️ El formulario de 2022 no tiene columna 'Permanent Plot', se crea vacía.
🌳 Doyle calculado

✅ No se imputaron árboles
✔️ Filas válidas: 1407 | ❌ Filas con error: 0


In [22]:
from core.schema_helpers import rename_columns_using_schema, get_dtypes_for_dataframe, FINAL_ORDER
from CruisesProcessor.general_importer import prepare_df_for_sql, save_inventory_to_sql, ensure_table

# 1. Renombra columnas usando schema (si hace falta)
df_sql = rename_columns_using_schema(df_good)

# 2. Alinea y rellena columnas según el orden oficial
for col in FINAL_ORDER:
    if col not in df_sql.columns:
        df_sql[col] = pd.NA
df_sql = df_sql[FINAL_ORDER]

# 3. Castea los tipos correctos
dtypes = get_dtypes_for_dataframe(df_sql)
for col, dtype in dtypes.items():
    if col in df_sql.columns:
        try:
            df_sql[col] = df_sql[col].astype(dtype.python_type, errors="ignore")
        except Exception:
            pass

# 4. Limpia NAs para SQL (usar clean_for_sql si quieres)
df_sql = df_sql.astype(object).where(df_sql.notna(), None)

# 5. Asegura la tabla y mete los datos
ensure_table(df_sql, engine, f"inventory_{specific_file}_2022", recreate=False)
save_inventory_to_sql(
    df_sql,
    engine,
    f"inventory_{specific_file}_2022",
    if_exists="append",
    dtype=dtypes,
    progress=True
)


🔑 PRIMARY KEY agregada en 'id'

=== INICIO DE IMPORTACIÓN ===
💻 Conectado a la base de datos helloworldtree


Insertando → inventory_mx_2022: 100%|██████████| 2/2 [00:00<00:00,  6.38filas/s]

✅ Bulk insert completado: 
 'inventory_mx_2022' (1407 filas)
